In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import os
import time
from random import randint
from fuzzywuzzy import process

/home/frostburn/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def get_matches(query,choices,limit=1):
    results = process.extract(query,[choices],limit=limit)
    return results

In [3]:
print("download.default_directory=" + os.getcwd() + "/Downloads")

download.default_directory=/home/frostburn/Desktop/Web-Scraping/ResearchGate/Downloads


In [4]:
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": os.getcwd() + "/Downloads",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "headless":True
}
options.add_experimental_option('prefs', prefs)

In [5]:
start_page = "https://www.researchgate.net/search/authors?q"


Process Flow
-------------------
Search professors from start_page by name
Click Search
Find the first reseach item in the research section
Determine type of publication
Get Abstract if Abstract exists, if not skip to next
Click citation download link
Select option (1 = RIS, 2 = BibTex, 3 = Plain text)
Select option  (1 = citation 2 = citation and abstract)
Download
Go to most recent downloaded file
Read text from that file
Place citation into variable
If abstract exists for citation place into abstract variable if no abstract found insert "No abstract found."

In [6]:
def research_gate_rerunner(record,driver,j):

    search_bar = driver.find_element_by_tag_name('input')
    search_bar.clear()
    search_bar.send_keys("\"" + record.faculty.strip() + "\" CSUN")
    search_bar.send_keys(Keys.ENTER)
    results = False

    if driver.find_elements_by_xpath('//*[@class="nova-o-stack__item"]/div'):
        results = True
    else:
        search_bar = driver.find_element_by_tag_name('input')
        search_bar.clear()
        search_bar.send_keys("\"" + record.faculty.strip() + "\"" + " Northridge")
        search_bar.send_keys(Keys.ENTER)
        if driver.find_elements_by_xpath('//*[@class="nova-o-stack__item"]/div'):
            results = True

    if results:

        results = False

        cards = driver.find_elements_by_xpath('//*[@class="nova-c-card__body nova-c-card__body--spacing-inherit"]')
        profile_links = driver.find_elements_by_link_text('View Profile')

        y = 0

        for card in cards:

            card = card.get_attribute('innerHTML')
            card_soup = BeautifulSoup(card, 'html.parser')

            name = card_soup.find('a',class_='nova-e-link nova-e-link--color-inherit nova-e-link--theme-bare').text
            details = card_soup.find_all('li',class_='nova-e-list__item nova-v-person-item__info-section-list-item')

            if len(details) > 1:
                institution = details[0].text
                department = details[1].text
            elif len(details) > 0:
                institution = details[0].text
                department = ""
            elif len(details) == 0:
                instution = ""
                department = ""

            name_match = get_matches(name,record.faculty)
            print(name)
            institution_match = get_matches(institution,'California State University, Northridge')
            print(institution)
            department_match = get_matches(department,record.department)
            print(department)

            if (name_match[0][1] >= 85) and (institution_match[0][1] >= 90) and (department_match[0][1] >= 90):
                url = profile_links[y].get_attribute('href')
                results = True
            if results:
                 break    
            y += 1

    if results:
        citation_collector(record,url,j)


In [7]:
def citation_collector(record,url,j):
    
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(5)
    driver.get(url)

    #Attempt to find the total number of research items for a given professsor 
    num_research_items = driver.find_element_by_xpath('//div[@class="nova-e-text nova-e-text--size-xl nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit"]')
    num_research_items = num_research_items.text
    num_research_items = int(num_research_items)
    print(num_research_items)

    # Retrieve all the urls of the first page
    research_a_tags = driver.find_elements_by_xpath('//div[@id="research"]//div/a[@class="nova-e-link nova-e-link--color-inherit nova-e-link--theme-bare"]')
    urls = [tag.get_attribute("href") for tag in research_a_tags]
    titles = [tag.text for tag in research_a_tags]
    paper_types = driver.find_elements_by_xpath('//div[@id="research"]//a[@class="nova-e-badge nova-e-badge--color-green nova-e-badge--display-block nova-e-badge--luminosity-high nova-e-badge--size-l nova-e-badge--theme-solid nova-e-badge--radius-m nova-v-publication-item__type"]')
    paper_types = [item.text for item in paper_types]
    num_research_items -= 100

    current_url = driver.current_url

    page_count = 2
    while num_research_items > 0:
        driver.get(current_url + '/' + str(page_count))
        temp_tags = driver.find_elements_by_xpath('//div[@id="research"]//div/a[@class="nova-e-link nova-e-link--color-inherit nova-e-link--theme-bare"]')
        for tag in temp_tags:
            urls.append(tag.get_attribute("href"))
            titles.append(tag.text)
        temp_papers = driver.find_elements_by_xpath('//div[@id="research"]//a[@class="nova-e-badge nova-e-badge--color-green nova-e-badge--display-block nova-e-badge--luminosity-high nova-e-badge--size-l nova-e-badge--theme-solid nova-e-badge--radius-m nova-v-publication-item__type"]')
        for paper in temp_papers:
            paper_types.append(paper.text)
        num_research_items -= 100
        page_count += 1

    while "Full-text available" in paper_types:
        paper_types.remove("Full-text available")

    for i in range(j,len(urls)):
        
        print(i)

        citation_text = ""
        title = ""
        abstract = ""
        book_title = ""
        journal_title = ""
        date = ""
        collabs = ""
        publisher = ""
        edition = ""
        series = ""
        volume = ""
        issue = ""
        chapter = ""
        pages = ""
        doi = ""
        issn = ""
        isbn = ""
        handle = ""

        #time.sleep(randint(5,10))
        driver.get(urls[i])

        # check for the DOI
        if driver.find_elements_by_xpath('//*[@id="lite-page"]/main/section/section[1]/div[1]/div[2]'):
            # Grab the DOI
            meta = driver.find_element_by_xpath('//*[@id="lite-page"]/main/section/section[1]/div[1]/div[2]')
            doi = meta.text.split(" ")
            if len(doi) > 1:
                if '\n' in doi[1]:
                    doi = doi[1].split("\n")
                    doi = doi[0]
                else:
                    doi = doi[1]
            else:
                doi

        # check for the abstract
        if driver.find_elements_by_xpath('//div[@class="nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-auto nova-e-text--color-inherit"]'):
            # get the abstract
            abstract = driver.find_element_by_xpath('//div[@class="nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-auto nova-e-text--color-inherit"]')    
            abstract = abstract.text

        # then we'll get the collaborators
        collab = driver.find_elements_by_xpath('html/body/div[2]/main/section/section/ul/li/div/div/div/div/div[2]/div/div/div/div/div/a')
        collabs = ""
        for person in collab:
            collabs += person.text


        # Attempt to get the date
        if driver.find_elements_by_xpath('html/body/div[2]/main/section/section/div/div/span[2]'):
            date = driver.find_element_by_xpath('html/body/div[2]/main/section/section/div/div/span[2]')
            date = date.text

        #download the citatation
        #//*[@id="lite-page"]/main/section/section[1]/div[1]/a
        download_link = driver.find_element_by_xpath('//*[@id="lite-page"]/main/section/section/div/a')
        download_link.click()

        # click the text button
        text_radio_btn = driver.find_element_by_xpath("//span[contains(.,' Plain Text')]")
        text_radio_btn.click()

        # download the citation file
        download = driver.find_element_by_xpath('//button[@class="nova-c-button nova-c-button--align-center nova-c-button--radius-m nova-c-button--size-m nova-c-button--color-blue nova-c-button--theme-solid nova-c-button--width-auto"]')
        download.click()

        time.sleep(3)

        #driver.close()


        # now that file is downloaded, need to open the file

        # here I grab the file name; the trick to this is making sure that its the only file in downloads
        citation_file = os.listdir(os.getcwd() + "/Downloads")[0]

        # grab the citation
        # i can probably get the name of the journal from the citation if thats something thats all over
        f = open("Downloads/"+citation_file, "r")
        citation_text = f.read()
        f.close()
        # delete it
        os.remove("Downloads/"+citation_file)

        citation_text


        pieces = citation_text.split('.')
        parts = []
        for piece in pieces:
            parts.append(piece.strip())
        #print(parts[3],",",parts[4])

        citations = open("research_gate_citations.csv", "a")
        citations.write(record.faculty.strip() + "|" + citation_text + "|" + titles[i] + "|" + paper_types[i] + "|")
        citations.write(abstract + "|" + book_title + "|" + journal_title + "|" + date + "|")
        citations.write(collabs + "|" + publisher + "|" + edition + "|" + series + "|")
        citations.write(volume + "|" + issue + "|" + chapter + "|" + pages + "|")
        citations.write(doi + "|" + issn + "|" + isbn + "|" + handle + "|" + url + "\n")
        citations.close()
        i+=1
    driver.close()
            

### The goal is to run all_faculty.csv through this scraper

In [8]:
all_faculty = pd.read_csv("../Data Sets/all_faculty.csv")
print(all_faculty.shape[0])
print(all_faculty.columns)
all_faculty.head()

2579
Index(['faculty', 'college', 'department', 'url'], dtype='object')


,faculty,college,department,url
0,Keji Chen,College of Business and Economics,Department of Accounting,https://www.csun.edu/acctis/faculty-and-staff-...
1,Michael E. Doron,College of Business and Economics,Department of Accounting,https://www.csun.edu/acctis/faculty-and-staff-...
2,Rafi Efrat,College of Business and Economics,Department of Accounting,https://www.csun.edu/acctis/faculty-and-staff-...
3,Monica Gianni,College of Business and Economics,Department of Accounting,https://www.csun.edu/acctis/faculty-and-staff-...
4,Young-Won Her,College of Business and Economics,Department of Accounting,https://www.csun.edu/acctis/faculty-and-staff-...


In [9]:
all_faculty.iloc[2134]

faculty                                      Gilberto E. Flores
college                                        Science and Math
department                                Department of Biology
url            https://www.csun.edu/science-mathematics/biol...
Name: 2134, dtype: object

In [27]:
start = "https://www.researchgate.net/search/authors?q"
try:
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(5)
    driver.get(start)
    research_gate_rerunner(all_faculty.iloc[2495], driver, 1)
finally:
    #print('Hello World')
    driver.close()

Teri Berkel
California State University, Northridge
Department of Kinesiology
3
1
2


IndexError: list index out of range

In [ ]:
1355 Deqing Ren 118/707
2123 Larry Allen 118/124
2132 Peter Edmunds 57/225